In [1]:
# 멀티 GPU 관련 코드라서 지우고 돌리시면 됩니다
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### lenet

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras

from keras import models, layers

2024-05-09 14:17:53.261534: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 14:17:53.261575: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 14:17:53.261617: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 14:17:53.268977: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 14:17:54.050636: W tensorflow/compiler/

In [3]:
train_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-balanced-train.csv')
test_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-balanced-test.csv')

In [4]:
print(train_data.shape)
print(test_data.shape)

(112799, 785)
(18799, 785)


In [5]:
y_train = train_data.iloc[:,:1]
x_train = train_data.iloc[:,1:]

y_test = test_data.iloc[:,:1]
x_test = test_data.iloc[:,1:]

In [6]:
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()

y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [7]:
x_train_np = np.reshape(x_train_np, newshape = (112799, 28, 28, 1))
x_test_np = np.reshape(x_test_np, newshape = (18799, 28, 28, 1))

y_train_np = tf.keras.utils.to_categorical(y_train_np,47)
y_test_np = tf.keras.utils.to_categorical(y_test_np,47)

In [8]:
model_lenet5 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5,
                    strides=1, activation='tanh',
                    input_shape=[28,28,1], padding='same'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5,
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5,
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(47, activation = "softmax")
])
model_lenet5.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

2024-05-09 14:17:58.622728: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 14:17:58.652118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 14:17:58.652394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
model_lenet5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (Average  (None, 14, 14, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [10]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [11]:
model_lenet5.fit(x_train_np, y_train_np, batch_size=300, epochs = 50, callbacks=[callbacks])

Epoch 1/50


2024-05-09 14:18:05.471360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2024-05-09 14:18:08.068733: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x142c4220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-09 14:18:08.068765: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-05-09 14:18:08.076652: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-09 14:18:08.174867: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


376/376 [==============================] - 11s 7ms/step - loss: 1.0414 - accuracy: 0.7031
Epoch 2/50
376/376 [==============================] - 2s 5ms/step - loss: 0.4994 - accuracy: 0.8339
Epoch 3/50
376/376 [==============================] - 2s 6ms/step - loss: 0.4270 - accuracy: 0.8522
Epoch 4/50
376/376 [==============================] - 2s 6ms/step - loss: 0.3891 - accuracy: 0.8636
Epoch 5/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3634 - accuracy: 0.8714
Epoch 6/50
376/376 [==============================] - 2s 6ms/step - loss: 0.3463 - accuracy: 0.8760
Epoch 7/50
376/376 [==============================] - 2s 5ms/step - loss: 0.3322 - accuracy: 0.8808
Epoch 8/50
376/376 [==============================] - 2s 6ms/step - loss: 0.3225 - accuracy: 0.8819
Epoch 9/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3091 - accuracy: 0.8873
Epoch 10/50
376/376 [==============================] - 2s 6ms/step - loss: 0.2989 - accuracy: 0.8895
Epoch 11/

In [12]:
loss_lenet, acc_lenet = model_lenet5.evaluate(x_test_np,y_test_np)

588/588 [==============================] - 2s 3ms/step - loss: 0.5557 - accuracy: 0.8649


### Resnet

In [13]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# 데이터 전처리
x_train_resized = tf.image.resize(x_train_np, [32, 32])  # ResNet에 맞게 크기 조정
x_test_resized = tf.image.resize(x_test_np, [32, 32])

# 이미지 채널을 3개로 조정 (RGB)
x_train_rgb = tf.repeat(x_train_resized, 3, axis=3)
x_test_rgb = tf.repeat(x_test_resized, 3, axis=3)

# ResNet-50 모델 정의
input_tensor = Input(shape=(32, 32, 3))
base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(47, activation='softmax')(x)
model_resnet = Model(inputs=base_model.input, outputs=output)

In [14]:
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model_resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [16]:
model_resnet.fit(x_train_rgb, y_train_np, batch_size=1000, epochs=50, callbacks=[callbacks])

Epoch 1/50
113/113 [==============================] - 37s 115ms/step - loss: 1.6644 - accuracy: 0.5888
Epoch 2/50
113/113 [==============================] - 10s 89ms/step - loss: 0.4473 - accuracy: 0.8427
Epoch 3/50
113/113 [==============================] - 10s 90ms/step - loss: 0.3496 - accuracy: 0.8724
Epoch 4/50
113/113 [==============================] - 10s 91ms/step - loss: 0.3063 - accuracy: 0.8842
Epoch 5/50
113/113 [==============================] - 10s 91ms/step - loss: 0.2708 - accuracy: 0.8958
Epoch 6/50
113/113 [==============================] - 10s 90ms/step - loss: 0.2545 - accuracy: 0.9014
Epoch 7/50
113/113 [==============================] - 11s 93ms/step - loss: 0.2434 - accuracy: 0.9047
Epoch 8/50
113/113 [==============================] - 10s 91ms/step - loss: 0.2307 - accuracy: 0.9090
Epoch 9/50
113/113 [==============================] - 11s 94ms/step - loss: 0.2223 - accuracy: 0.9118
Epoch 10/50
113/113 [==============================] - 10s 91ms/step - loss: 0.21

In [17]:
loss_resnet, acc_resnet = model_resnet.evaluate(x_test_rgb,y_test_np)

588/588 [==============================] - 7s 10ms/step - loss: 0.5926 - accuracy: 0.8576


In [18]:
results = {"Model": ["lenet5", "resnet-50"], "Loss": [loss_lenet, loss_resnet], "Accuracy": [acc_lenet, acc_resnet]}
results_df = pd.DataFrame(results)

import IPython
display(results_df)

,Model,Loss,Accuracy
0,lenet5,0.555697,0.864940
1,resnet-50,0.592595,0.857599
